<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/retrievers/vectara_auto_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 从Vectara索引中自动检索

本指南展示了如何在LlamaIndex中使用Vectara执行**自动检索**。

通过自动检索，我们在将查询提交给Vectara之前解释检索查询，以识别查询的潜在重写，将其作为一个较短的查询以及一些元数据过滤一起提交。

例如，像“2022年的收入是多少”的查询可能被重写为“收入是多少”，并附带一个“doc.year = 2022”的过滤器。让我们通过一个示例来看看这是如何工作的。


## 设置


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
!pip install llama_index llama-index-llms-openai llama-index-indices-managed-vectara

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
from llama_index.core.schema import TextNode
from llama_index.core.indices.managed.types import ManagedIndexQueryMode
from llama_index.indices.managed.vectara import VectaraIndex
from llama_index.indices.managed.vectara import VectaraAutoRetriever

from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo

from llama_index.llms.openai import OpenAI

## 定义一些样本数据

首先，我们定义了一个电影数据集：
1. 每个节点描述了一部电影。
2. `text` 描述了电影本身，而 `metadata` 定义了一些元数据字段，比如年份、导演、评分或类型。

在Vectara中，您需要在您的语料库中 [定义](https://docs.vectara.com/docs/learn/metadata-search-filtering/filter-overview) 这些元数据字段作为可过滤的属性，以便可以对它们进行过滤。


In [ ]:
nodes = [
    TextNode(
        text=(
            "A pragmatic paleontologist touring an almost complete theme park on an island "
            + "in Central America is tasked with protecting a couple of kids after a power "
            + "failure causes the park's cloned dinosaurs to run loose."
        ),
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    TextNode(
        text=(
            "A thief who steals corporate secrets through the use of dream-sharing technology "
            + "is given the inverse task of planting an idea into the mind of a C.E.O., "
            + "but his tragic past may doom the project and his team to disaster."
        ),
        metadata={
            "year": 2010,
            "director": "Christopher Nolan",
            "rating": 8.2,
        },
    ),
    TextNode(
        text="Barbie suffers a crisis that leads her to question her world and her existence.",
        metadata={
            "year": 2023,
            "director": "Greta Gerwig",
            "genre": "fantasy",
            "rating": 9.5,
        },
    ),
    TextNode(
        text=(
            "A cowboy doll is profoundly threatened and jealous when a new spaceman action "
            + "figure supplants him as top toy in a boy's bedroom."
        ),
        metadata={"year": 1995, "genre": "animated", "rating": 8.3},
    ),
    TextNode(
        text=(
            "When Woody is stolen by a toy collector, Buzz and his friends set out on a "
            + "rescue mission to save Woody before he becomes a museum toy property with his "
            + "roundup gang Jessie, Prospector, and Bullseye. "
        ),
        metadata={"year": 1999, "genre": "animated", "rating": 7.9},
    ),
    TextNode(
        text=(
            "The toys are mistakenly delivered to a day-care center instead of the attic "
            + "right before Andy leaves for college, and it's up to Woody to convince the "
            + "other toys that they weren't abandoned and to return home."
        ),
        metadata={"year": 2010, "genre": "animated", "rating": 8.3},
    ),
]

然后我们将样本数据加载到Vectara索引中。


In [ ]:
import os

os.environ["VECTARA_API_KEY"] = "<YOUR_VECTARA_API_KEY>"
os.environ["VECTARA_CORPUS_ID"] = "<YOUR_VECTARA_CORPUS_ID>"
os.environ["VECTARA_CUSTOMER_ID"] = "<YOUR_VECTARA_CUSTOMER_ID>"

index = VectaraIndex(nodes=nodes)

## 定义 `VectorStoreInfo`

我们定义了一个 `VectorStoreInfo` 对象，其中包含了我们的 Vectara 索引支持的元数据过滤器的结构化描述。这些信息随后被用于自动检索提示，使得 LLM 能够推断用于特定查询的元数据过滤器。


In [ ]:
vector_store_info = VectorStoreInfo(    content_info="关于电影的信息",    metadata_info=[        MetadataInfo(            name="genre",            description="""                电影的类型。                可选值为 ['science fiction', 'fantasy', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']            """,            type="字符串",        ),        MetadataInfo(            name="year",            description="电影上映的年份",            type="整数",        ),        MetadataInfo(            name="director",            description="电影导演的姓名",            type="字符串",        ),        MetadataInfo(            name="rating",            description="电影的评分，范围为1-10",            type="浮点数",        ),    ],)

## 运行自动检索
现在让我们创建一个`VectaraAutoRetriever`实例并尝试`retrieve()`：


In [ ]:
from llama_index.indices.managed.vectara import VectaraAutoRetriever
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

retriever = VectaraAutoRetriever(
    index,
    vector_store_info=vector_store_info,
    llm=llm,
    verbose=True,
)

In [ ]:
retriever.retrieve("movie directed by Greta Gerwig")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Using query str: movie directed by Greta Gerwig
Using implicit filters: [('director', '==', 'Greta Gerwig')]
final filter string: (doc.director == 'Greta Gerwig')


[NodeWithScore(node=TextNode(id_='935c2319f66122e1c4189ccf32164b362d4b6bc2af4ede77fc47fd126546ba8d', embedding=None, metadata={'lang': 'eng', 'offset': '0', 'len': '79', 'year': '2023', 'director': 'Greta Gerwig', 'genre': 'fantasy', 'rating': '9.5'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Barbie suffers a crisis that leads her to question her world and her existence.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.61976165)]

In [ ]:
retriever.retrieve("a movie with a rating above 8")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Using query str: movie with rating above 8
Using implicit filters: [('rating', '>', 8)]
final filter string: (doc.rating > '8')


[NodeWithScore(node=TextNode(id_='a55396c48ddb51e593676d37a18efa8da1cbab6fd206c9029ced924e386c12b5', embedding=None, metadata={'lang': 'eng', 'offset': '0', 'len': '129', 'year': '1995', 'genre': 'animated', 'rating': '8.3'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="A cowboy doll is profoundly threatened and jealous when a new spaceman action figure supplants him as top toy in a boy's bedroom.", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.5414715),
 NodeWithScore(node=TextNode(id_='b1e87de59678f3f1831327948a716f6e5a217c9b8cee4c08d5555d337825fca8', embedding=None, metadata={'lang': 'eng', 'offset': '0', 'len': '220', 'year': '2010', 'director': 'Christopher Nolan', 'rating': '8.2'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='A thief who steals corporate secrets through the use of dream-s

我们还可以在`VectaraAutoRetriever`中包含标准的`VectaraRetriever`参数。例如，如果我们想要包含一个`filter`，该`filter`将被添加到查询本身的任何额外过滤中，我们可以按照以下方式操作：
